In [39]:
import pandas as pd

In [40]:
def code_prefix(x):
    if len(str(x)) == 3:
        return 'ms-drg'
    elif len(str(x)) == 5:
        return 'hcpcs_cpt'
    else:
        return 'none'
    
def payer_category(x):
    payers = {
        'PRICING_AMT': 'gross',
        'Self Pay/Uninsured': 'cash',
        'Pricing Max': 'max',
        'Pricing Min': 'min'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [41]:
file = '59-1943502_ShandsTeachingHospitalandClinics_standardcharges (3).csv'
df = pd.read_csv(file, dtype=str)

In [42]:
df.rename(columns={
    'PROCEDURE': 'internal_code',
    'UNIT': 'ndc_quantity_desc',
    'SUF EAP DESC': 'description',
    'CODE': 'code_orig'
},
    inplace=True)

In [43]:
df.drop('Eff Date', axis=1, inplace=True)

In [44]:
cols = df.columns.tolist()
id_vars = cols[:4]
val_vars = cols[4:]

df = pd.melt(df, id_vars=id_vars, value_vars=val_vars, value_name='rate', var_name='payer_orig')

In [45]:
df['code_prefix'] = df['code_orig'].apply(code_prefix)
df['payer_category'] = df['payer_orig'].apply(payer_category)

In [55]:
df.loc[df['ndc_quantity_desc'].isin(['1 each', 'Ea', 'DISCHARGE']), 'ndc_quantity_desc'] = pd.NA

In [59]:
df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')

C:\Users\adria\AppData\Local\Temp\ipykernel_9612\505826267.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')


In [61]:
df['file_last_updated'] = '2023-01-01'
df['hospital_ccn'] = '100113'
df['hospital_ein'] = '59-1943502'
df['url'] = 'https://ufhealth.org/sites/default/files/media/financial-assistance/59-1943502_ShandsTeachingHospitalandClinics_standardcharges%20%283%29.csv'
df['filename'] = file
df['payer_name'] = df['payer_orig']

In [64]:
df.to_csv('shands.csv', index=False)

,internal_code,ndc_quantity_desc,code_orig,description,payer_orig,rate,code_prefix,payer_category,file_last_updated,hospital_ccn,hospital_ein,url,filename,payer_name
916741,65500273,<NA>,C1769,UF CF G-WIRE ANGIO ROSEN CVD TIP .035X18,Self Pay/Uninsured,17.55,none,cash,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,Self Pay/Uninsured
263261,37500099,<NA>,82360,HB CALCULUS ASSAY QUANT,FRINGE BENEFIT COORDINATORS,57.00,hcpcs_cpt,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,FRINGE BENEFIT COORDINATORS
127427,4422,<NA>,65000002,LEVOTHYROXINE SODIUM 75 MCG PO TABS,CHOICE CARE NETWORK,8.09,none,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,CHOICE CARE NETWORK
302979,10470,<NA>,65000002,LOVASTATIN 20 MG PO TABS,HEALTH FIRST,10.98,none,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,HEALTH FIRST
565103,37560208,<NA>,83520,HB ANTI-MULLERIAN HORMONE,MEDICARE CARE PLUS,Bundled,hcpcs_cpt,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,MEDICARE CARE PLUS
916782,65500314,<NA>,C1874,UF CF STENT GRFT VIABAHN VBH 6X10X120(VB,Self Pay/Uninsured,3435.75,none,cash,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,Self Pay/Uninsured
81871,15862,1000 mL,65000008,DEXTROSE-NACL 2.5-0.45 % IV SOLN,AVMED,22.68,none,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,AVMED
657916,27500190,<NA>,28190,HB REMOVE FB FOOT SUBQ,MEDICARE OPTIMUM,654.61,hcpcs_cpt,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,MEDICARE OPTIMUM
673625,25060040,<NA>,90935,CF HB HEMODIALYSIS,MEDICARE UNITED HEALTHCARE,668.91,hcpcs_cpt,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,MEDICARE UNITED HEALTHCARE
896169,32500719,<NA>,78660,HB RADIONUC DACROCYSTOGRAM,WC HERITAGE SUMMIT,810.00,hcpcs_cpt,payer,2023-01-01,100113,59-1943502,https://ufhealth.org/sites/default/files/media...,59-1943502_ShandsTeachingHospitalandClinics_st...,WC HERITAGE SUMMIT
